# Indexe in Apache Pinot
## Schema and Table Generation 

In [4]:
!pip install pandas
import pandas
import json
import requests
import copy
import time

In [6]:
schemaConfiguration = {
  "schemaName": "trips2",
  "dimensionFieldSpecs": [
    {
      "name": "rider_name",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
    {
      "name": "driver_name",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
    {
      "name": "license_plate",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
    {
      "name": "start_location",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
    {
      "name": "start_zip_code",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
     {
      "name": "start_location_state",
      "dataType": "STRING",
      "defaultNullValue": ""
    }, 
    {
      "name": "end_location",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
    {
      "name": "end_zip_code",
      "dataType": "STRING",
      "defaultNullValue": ""
    },
      {
      "name": "end_location_state",
      "dataType": "STRING",
      "defaultNullValue": ""
    }, 
    {
      "name": "rider_is_premium",
      "dataType": "INT",
      "defaultNullValue": 0
    }
  ],
  "metricFieldSpecs": [
    {
      "name": "count",
      "dataType": "LONG",
      "defaultNullValue": 1
    },
    {
      "name": "payment_amount",
      "dataType": "FLOAT",
      "defaultNullValue": 0
    },
    {
      "name": "payment_tip_amount",
      "dataType": "FLOAT",
      "defaultNullValue": 0
    },
    {
      "name": "trip_wait_time_millis",
      "dataType": "LONG",
      "defaultNullValue": 0
    },
    {
      "name": "rider_rating",
      "dataType": "INT",
      "defaultNullValue": 0
    },
    {
      "name": "driver_rating",
      "dataType": "INT",
      "defaultNullValue": 0
    }
  ],
  "dateTimeFieldSpecs": [
    {
      "name": "trip_start_time_millis",
      "dataType": "LONG",
      "format": "1:MILLISECONDS:EPOCH",
      "granularity": "1:MINUTES",
      "dateTimeType": "PRIMARY"
    },
    {
      "name": "request_time_millis",
      "dataType": "LONG",
      "format": "1:MILLISECONDS:EPOCH",
      "granularity": "1:MINUTES",
      "dateTimeType": "SECONDARY"
    },
    {
      "name": "trip_end_time_millis",
      "dataType": "LONG",
      "format": "1:MILLISECONDS:EPOCH",
      "granularity": "1:MINUTES",
      "dateTimeType": "SECONDARY"
    }
  ]
}

# Create Schema
response = requests.post('http://pinot-controller.pinot:9000/schemas?override=false', json=schemaConfiguration)
print("Create Schema: " + response.text)
# Get Schema
response = (requests.get('http://pinot-controller.pinot:9000/schemas')).json()
print("Get all schemas: " + str(response))

Create Schema: {"code":500,"error":"Failed to add new schema trips2."}
Get all schemas: ['trips', 'trips2']


Different tables are created with different index configurations. These configurations are applied to the tableIndexConfig-section of the general json_tableConfig object. If a table has been created, its name and a description of the defined index is added to an array. This array contains all created tables with their index details. It will be used later to execute the same query on each table.
The tables defined in the following are realtime tables consuming from the same Kafka topic. Therefore, all of them will contain the same data records. To ensure, that the consumption of the tables has finished, we are monitoring when streaming data was ingested the last time to the table. When a query is sent to the Broker Query API Endpoint, the response will return the result records, but also other paramters. Paramter minConsumingFreshnessTimeMs describes when the last ingestion took place. For the following use case we defined, that if there hasn't been any new data consumption for the last 15 seconds, the before generated data records of the Kafka topic have been consumed and there are enough data records to proceed with the analyzing the impact of different indices.
(Remark: We are only checking the last updated time stamp for one table and assume, that their is not a big time deviation for the different tables.)

In [116]:
table_list = []

json_tableConfig = {
  "tableName": "variable_tableName",
  "tableType": "REALTIME",
  "segmentsConfig": {
    "timeColumnName": "trip_start_time_millis",
    "timeType": "MILLISECONDS",
    "retentionTimeUnit": "DAYS",
    "retentionTimeValue": "60",
    "schemaName": "trips2",
    "replication": "1",
    "replicasPerPartition": "1"
  },
  "tenants": {},
  "tableIndexConfig": {
    "loadMode": "MMAP",
    "streamConfigs": {
      "streamType": "kafka",
      "stream.kafka.consumer.type": "simple",
      "stream.kafka.topic.name": "trips_gendata2",
      "stream.kafka.decoder.class.name": "org.apache.pinot.plugin.stream.kafka.KafkaJSONMessageDecoder",
      "stream.kafka.consumer.factory.class.name": "org.apache.pinot.plugin.stream.kafka20.KafkaConsumerFactory",
      "stream.kafka.zk.broker.url": "pinot-kafka-zookeeper:2181",
      "stream.kafka.broker.list": "pinot-kafka:9092",
      "realtime.segment.flush.threshold.time": "12h",
      "realtime.segment.flush.threshold.size": "1000",
      "stream.kafka.consumer.prop.auto.offset.reset": "smallest"
    }
  },
  "metadata": {
    "customConfigs": {}
  }
} 

def createTable(newTable_name, index_text, tableconfig_json):
    #Input: Name of new table, index description, table configuration in json structure
    response = requests.post('http://pinot-controller.pinot:9000/tables', json=tableconfig_json)
    print(response)
    print(response.text)
    table_list.append([newTable_name, index_text])


execution_start_time = int(round(time.time() * 1000))

# Create a new table with default index for each column (no configuration required)
newTable_defaultIndex = copy.deepcopy(json_tableConfig)
newTable_defaultIndex["tableName"] = "trips_default_index"
createTable(newTable_defaultIndex["tableName"], 'Default Index (Dictionary-encoded forward index with bit compression) for each column', newTable_defaultIndex)

# Create a new table with raw value forward index
newTable_rawForwardIndex = copy.deepcopy(json_tableConfig)
newTable_rawForwardIndex["tableName"] = "trips_rawForwardIndex"
newTable_rawForwardIndex["tableIndexConfig"]["noDictionaryColumns"] = ["start_location"]
createTable(newTable_rawForwardIndex["tableName"], 'Raw value forward index on start_location',newTable_rawForwardIndex)


# Create a new table with sorted forward index with run-length encoding
newTable_sortedForwardIndex = copy.deepcopy(json_tableConfig)
newTable_sortedForwardIndex["tableName"] = "trips_sortedForwardIndex"
newTable_sortedForwardIndex["tableIndexConfig"]["sortedColumn"] = ["start_location"]
createTable(newTable_sortedForwardIndex["tableName"],'Sorted forward index with run-length encoding on start location',newTable_sortedForwardIndex)


# Create a new table with bitmap inverted index
newTable_bitmapInvertedIndex2 = copy.deepcopy(json_tableConfig)
newTable_bitmapInvertedIndex2["tableName"] = "trips_bitmapInvertedIndex_startLocation"
newTable_bitmapInvertedIndex2["tableIndexConfig"]["invertedIndexColumns"] = ["start_location"]
createTable(newTable_bitmapInvertedIndex2["tableName"],'Bitmap inverted index on start_location',newTable_bitmapInvertedIndex2)


# Create a new table with sorted inverted index
newTable_sortedInvertedIndex2 = copy.deepcopy(json_tableConfig)
newTable_sortedInvertedIndex2["tableName"] = "trips_sortedInvertedIndex_startLocation"
newTable_sortedInvertedIndex2["tableIndexConfig"]["invertedIndexColumns"] = ["start_location"]
newTable_sortedInvertedIndex2["tableIndexConfig"]["sortedColumn"] = ["start_location"]
createTable(newTable_sortedInvertedIndex2["tableName"],'Sorted inverted index on start_location',newTable_sortedInvertedIndex2)

# Create a new table with star tree  index
newTable_starTree1 = copy.deepcopy(json_tableConfig)
newTable_starTree1["tableName"] = "trips_starTreeIndex"
newTable_starTree1["tableIndexConfig"]["starTreeIndexConfigs"] = [{
    "dimensionsSplitOrder": [
      "rider_is_premium",
      "start_location_state",
      "end_location"
    ],
    "skipStarNodeCreationForDimensions": [
    ],
    "functionColumnPairs": [
      "SUM__payment_amount",
        "AVG__driver_rating"
    ],
    "maxLeafRecords": 1
  }]
createTable(newTable_starTree1["tableName"],'Star Tree 1',newTable_starTree1)



# Create a new table with star tree  index
newTable_starTree11 = copy.deepcopy(json_tableConfig)
newTable_starTree11["tableName"] = "trips_starTreeIndex1"
newTable_starTree11["tableIndexConfig"]["starTreeIndexConfigs"] = [{
    "dimensionsSplitOrder": [
      "rider_is_premium",
      "start_location_state",
      "end_location"
    ],
    "skipStarNodeCreationForDimensions": [
    ],
    "functionColumnPairs": [
      "SUM__payment_amount"
    ],
    "maxLeafRecords": 1
  }]
createTable(newTable_starTree11["tableName"],'Star Tree 1.1',newTable_starTree11)


# Create a new table with star tree  index
newTable_starTree2 = copy.deepcopy(json_tableConfig)
newTable_starTree2["tableName"] = "trips_starTreeIndex2"
newTable_starTree2["tableIndexConfig"]["starTreeIndexConfigs"] = [{
    "dimensionsSplitOrder": [
      "rider_is_premium",
      "start_location_state",
      "rider_rating"
    ],
    "skipStarNodeCreationForDimensions": [
    ],
    "functionColumnPairs": [
      "SUM__payment_amount"
    ],
    "maxLeafRecords": 4
  }]
createTable(newTable_starTree2["tableName"],'Star Tree 2',newTable_starTree2)


# Create a new table with star tree  index
newTable_starTree3 = copy.deepcopy(json_tableConfig)
newTable_starTree3["tableName"] = "trips_starTreeIndex3"
newTable_starTree3["tableIndexConfig"]["starTreeIndexConfigs"] = [{
    "dimensionsSplitOrder": [
      "start_location_state",
      "end_location_state"
    ],
    "skipStarNodeCreationForDimensions": [
    ],
    "functionColumnPairs": [
      "SUM__payment_amount"
    ],
    "maxLeafRecords": 1
  }]
createTable(newTable_starTree3["tableName"],'Star Tree 3',newTable_starTree3)

# Create a new table with text search support
newTable_textIndex = copy.deepcopy(json_tableConfig)
newTable_textIndex["tableName"] = "trips_textIndex"
newTable_textIndex["fieldConfigList"]= [
  {
     "name":"driver_name",
     "encodingType":"RAW",
     "indexType":"TEXT"
  },
  {
     "name":"rider_name",
     "encodingType":"RAW",
     "indexType":"TEXT"
  }
]
newTable_textIndex["tableIndexConfig"]["noDictionaryColumns"] = [
     "driver_name",
     "rider_name"
 ]
createTable(newTable_textIndex["tableName"],'Text Index',newTable_textIndex)

# The new created tables will consume all data records of the Kafka topic defined in the table configuration file. To ensure, that all current records of the topic have been consumed before 
# proceeding with the chapter about indexes, it will be displayed when there have been no new data consumption in the last 15 seconds for one table. 

tableConsuming = True
while tableConsuming:
    response = requests.post('http://pinot-broker.pinot:8099/query/sql', json={
            "sql" : "SELECT * FROM trips_default_index"
        })
    response_json=response.json()
    if response_json['minConsumingFreshnessTimeMs']<(int(round(time.time() * 1000)) - 15000 ) or response_json['minConsumingFreshnessTimeMs']>9000000036854775807:
        tableConsuming = False;
        print("")
        print("-- Consumption of generated data finished for trips_default_index --")


<Response [200]>
{"status":"Table trips_default_index_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_rawForwardIndex_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_sortedForwardIndex_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_bitmapInvertedIndex_startLocation_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_sortedInvertedIndex_startLocation_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_starTreeIndex_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_starTreeIndex1_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_starTreeIndex2_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_starTreeIndex3_REALTIME succesfully added"}
<Response [200]>
{"status":"Table trips_textIndex_REALTIME succesfully added"}

-- Consumption of generated data finished for trips_default_index --


### Table Size
Although the tables contain the same amount of records and also the same record values, the table size differs. This is because of the different indices. E.g. a text index on one column is more space intensive compared to the raw forward index. The Star-Tree Index is allocating much space, as it is based on pre-aggregation of columns.

In [120]:
for table in table_list:
    url_getsize = 'http://pinot-controller.pinot:9000/tables/' + table[0] + '_REALTIME/size?detailed=false'
    response = (requests.get(url_getsize)).json()
    print('tableName: ' + response['tableName'])
    print('Index Description: ' + table[1])
    v_size=response['reportedSizeInBytes']/1024/1024
    print('reportedSizeInMB: '+ str(v_size))
    print(" ")

tableName: trips_default_index_REALTIME
Index Description: Default Index (Dictionary-encoded forward index with bit compression) for each column
reportedSizeInMB: 54.57778263092041
 
tableName: trips_rawForwardIndex_REALTIME
Index Description: Raw value forward index on start_location
reportedSizeInMB: 47.64225101470947
 
tableName: trips_sortedForwardIndex_REALTIME
Index Description: Sorted forward index with run-length encoding on start location
reportedSizeInMB: 56.202518463134766
 
tableName: trips_bitmapInvertedIndex_startLocation_REALTIME
Index Description: Bitmap inverted index on start_location
reportedSizeInMB: 59.98648548126221
 
tableName: trips_sortedInvertedIndex_startLocation_REALTIME
Index Description: Sorted inverted index on start_location
reportedSizeInMB: 56.21041488647461
 
tableName: trips_starTreeIndex_REALTIME
Index Description: Star Tree 1
reportedSizeInMB: 142.4809045791626
 
tableName: trips_starTreeIndex1_REALTIME
Index Description: Star Tree 1.1
reportedSize

## Indexing - Comparison

Pinot uses the MYSQL_ANSI dialect. Executing Joins or nested Subqueries is not supported. 
For accessing multiple tables in queries, the query engine Presto is recommended. In this report, we will focus on the functionality Pinot offers.
Pinot doesn't support data definition language. As already seen, tables are created using the REST API.

Function executeSQLStatement takes a query as an input (table name is described with the variable XX) and will run this query on all tables which are defined in the table_list (created previously by the script). The top two records of the result data set will displayed once in total to have an insight into the result data set. Additionaly, the function will create a dataframe containing execution metrics about each query execution on one of the tables. Metrics of one query execution will only appended to the dataframe if no exeption occurs. 

In [84]:
def executeSQLStatement(sql_statement_with_variable):
    pandas.set_option('display.max_colwidth', None)
    df_metrics = pandas.DataFrame(columns=['indextype','table', 'numDocsScanned',
       'numEntriesScannedInFilter', 'numEntriesScannedPostFilter',
       'totalDocs', 'timeUsedMs',
       'minConsumingFreshnessTimeMs',
       'exceptions'])
    b_resultRecordsNotShown = True;
    for table in table_list:
    
        sql_statement = sql_statement_with_variable.replace("XX",table[0])
        response = requests.post('http://pinot-broker.pinot:8099/query/sql', json={
            "sql" : sql_statement
        })
        response_json=response.json()
        d = {'indextype': table[1], 'table': table[0],'numDocsScanned': [response_json['numDocsScanned']],'numDocsScanned': [response_json['numDocsScanned']],'numEntriesScannedInFilter': [response_json['numEntriesScannedInFilter']], 'numEntriesScannedPostFilter':[response_json['numEntriesScannedPostFilter']],'totalDocs':[response_json['totalDocs']],'timeUsedMs':[response_json['timeUsedMs']],'minConsumingFreshnessTimeMs':[response_json['minConsumingFreshnessTimeMs']],'exceptions':[response_json['exceptions']]}
        df_metrics_new = pandas.DataFrame(data=d)
        if not response_json['exceptions']:
             df_metrics = df_metrics.append(df_metrics_new,ignore_index=True)
       

        if b_resultRecordsNotShown:
            try:
                if not response_json['exceptions']:
                    columnNames = response_json['resultTable']['dataSchema']['columnNames']
                    rows = response_json['resultTable']['rows']

                    result_dataframe = pandas.DataFrame(columns=columnNames,data=rows)
                    print("Top two result records of: " + sql_statement )
                    display(result_dataframe.head(2))
                    b_resultRecordsNotShown = False
            except:
                pass

    print("Metrics of execution of: " + sql_statement_with_variable)
    display(df_metrics)


The most important metrics for our analysis are:
- timeUsedms: Total time between broker receiving request and sending response back to the client.
- numDocScanned: Number of documents/records scanned while query processing. (Includes records scanned in the filter phase as well as after applying the filter.)
- numEntriesScannedInFilter: If this number is high, applying an index on the selection criteria might improve performance.
- numEntriesScannedPostFilter: High number is an indicator for low selectivity. Instead of regular indices, a star-tree index could help.


The following indexes will be described with examples:

- Forward Index
    - __Default Index: Dictionary-encoded forward index with bit compression__: 
    Apache Pinot will use this index by default for each column if no other index is configured in the table configuration metadata. An id is assigned to each unique value of the column, afterwards a dictionary is build from the id to the value. In the forward index, only the bit-compressed id of is persisted instead of the values. This compression improves space efficiency of the storage, if there are few unique values.
    <img src="images/RawValueForwardIndex.png" width="35%" height="35%">
    - __Raw Value Forward Index__: A raw value forward index is configured as a noDictionaryColumn in the table configuration file. Values instead of dictionary ids will be stored. Because of that, no dictionary lookup is required and due to the locality of values the performance of scanning large number of values is improved. 
    - __Sorted forward index with run-length encoding__: The sorted forward index is applied on top of the dictionary-encoding. For each dictionary id, a start and end document id is stored. Only 1 sorted column can be created for each Pinot table.
- Inverted Index: Inverted Indexes reduce the number of records which need be processed by identifying the ones which contain the search term. The inverted index is created by selecting all unique values of the column. For each value, a list of document ids which contain the value will be assigned. If a column is used frequently for filtering, an inverted index will improve the performance. If we search e.g. for Hessen as a state, we can look up the inverted index for Hessen and identify the documents for which that value appears. 
    - __Bitmap inverted index__:
    - __Sorted inverted index__ A sorted index can benefit from data locality, but can only be applied to one column.
- __Star-Tree Index__: This index is built on multiple columns and pre-aggregates results, so that less values need to be processed. This can improve the query performance, on the other hand the pre-aggregation requires more storage (table size is about twice of the size as the other tables are).
- __Text Index__


### Default Index (Dictionary-encoded forward index with bit compression) vs Raw value forward index

In [83]:
executeSQLStatement("select count, driver_name, driver_rating, end_location, end_location_state from XX WHERE start_location='Steinau'")

Top two result records of: select count, driver_name, driver_rating, end_location, end_location_state from trips_default_index WHERE start_location='Steinau'


,count,driver_name,driver_rating,end_location,end_location_state
0,1,Elaine Corn,4,Böxlund,Schleswig-Holstein
1,1,Marie Patague,3,Eschwege,Hessen


Metrics of execution of: select count, driver_name, driver_rating, end_location, end_location_state from XX WHERE start_location='Steinau'


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,minConsumingFreshnessTimeMs,exceptions
0,Default Index (Dictionary-encoded forward index with bit compression) for each column,trips_default_index,17,17000,85,275000,11,9223372036854775807,[]
1,Raw value forward index on start_location,trips_rawForwardIndex,11,101000,55,275000,29,9223372036854775807,[]
2,Sorted forward index with run-length encoding on start location,trips_sortedForwardIndex,11,0,55,275004,11,9223372036854775807,[]
3,Bitmap inverted index on start_location,trips_bitmapInvertedIndex_startLocation,11,0,55,275004,8,9223372036854775807,[]
4,Sorted inverted index on start_location,trips_sortedInvertedIndex_startLocation,10,0,50,275004,10,9223372036854775807,[]
5,Star Tree 1,trips_starTreeIndex,10,0,50,275004,9,9223372036854775807,[]
6,Star Tree 2,trips_starTreeIndex2,11,10000,55,275004,9,9223372036854775807,[]
7,Text Index,trips_textIndex,11,10000,55,275000,25,9223372036854775807,[]


Comparison of table __trips_default_index (0)__ and table __trips_rawForwardIndex (1)__:

The query execution on table __trips_rawForwardIndex (1)__ takes more time. This is because more entries need to be scanned in the filter.  For table __trips_default_index__, less records are scanned. The main difference between the two index types is, that the index on column start_location of __trips_default_index (0)__ creates dictionaries. The dictionary provides compression when values of the columns occurr repeatedly. 
A dictionary index doesn't provide that advantage anymore, if a column consists of lots of unique values.

### Default Index (Dictionary-encoded forward index with bit compression) vs Sorted forward index with run-length encoding

In [43]:
executeSQLStatement("select SUM(payment_amount) from XX WHERE start_location='Heidelberg'")

Top two result records of: select SUM(payment_amount) from trips_default_index WHERE start_location='Heidelberg'


,sum(payment_amount)
0,54702.220154


Metrics of execution of: select SUM(payment_amount) from XX WHERE start_location='Heidelberg'


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,numGroupsLimitReached,totalDocs,timeUsedMs,segmentStatistics,minConsumingFreshnessTimeMs,numServersQueried,numServersResponded,numSegmentsQueried,numSegmentsProcessed,numSegmentsMatched,numConsumingSegmentsQueried,exceptions
0,Default Index (Dictionary-encoded forward index with bit compression) for each column,trips_default_index,129,96000,129,False,275000,12,[],9223372036854775807,1,1,276,275,96,1,[]
1,Raw value forward index on start_location,trips_rawForwardIndex,129,275000,129,False,275000,66,[],9223372036854775807,1,1,276,275,96,1,[]
2,Sorted forward index with run-length encoding on start location,trips_sortedForwardIndex,129,0,129,False,275004,13,[],9223372036854775807,1,1,277,276,96,1,[]
3,Bitmap inverted index,trips_bitmapInvertedIndex,129,96000,129,False,275000,14,[],9223372036854775807,1,1,276,275,96,1,[]
4,Bitmap inverted index on start_location,trips_bitmapInvertedIndex_startLocation,129,0,129,False,275004,7,[],9223372036854775807,1,1,277,276,96,1,[]
5,Sorted inverted index on end_location_state,trips_sortedInvertedIndex,129,96000,129,False,275000,8,[],9223372036854775807,1,1,276,275,96,1,[]
6,Sorted inverted index on start_location,trips_sortedInvertedIndex_startLocation,129,0,129,False,275004,9,[],9223372036854775807,1,1,277,276,96,1,[]
7,Star Tree 1,trips_starTreeIndex,129,0,129,False,275004,8,[],9223372036854775807,1,1,277,276,96,1,[]
8,Star Tree 2,trips_starTreeIndex2,129,96000,129,False,275004,11,[],9223372036854775807,1,1,277,276,96,1,[]
9,Text Index,trips_textIndex,129,96000,129,False,275000,29,[],9223372036854775807,1,1,276,275,96,1,[]


Comparison of __trips_default_index (0)__ and __trips_sortedInvertedIndex_startLocation (6)__:

The sorted forward index on column start_location of table __trips_sortedInvertedIndex_startLocation (6)__ has the advantage of data locality. Because of this, numEntriesScannedInFilter values 0.
Query execution time is faster using the sorted forward index on column start_location.

### Default Index (Dictionary-encoded forward index with bit compression) vs Inverted index (Bitmap + Sorted)

In [57]:
executeSQLStatement("select driver_name, rider_name from XX WHERE start_location='Frankfurt'")

Top two result records of: select driver_name, rider_name from trips_default_index WHERE start_location='Frankfurt'


,driver_name,rider_name


Metrics of execution of: select driver_name, rider_name from XX WHERE start_location='Frankfurt'


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,numGroupsLimitReached,totalDocs,timeUsedMs,segmentStatistics,minConsumingFreshnessTimeMs,numServersQueried,numServersResponded,numSegmentsQueried,numSegmentsProcessed,numSegmentsMatched,numConsumingSegmentsQueried,exceptions
0,Default Index (Dictionary-encoded forward index with bit compression) for each column,trips_default_index,0,0,0,False,275000,13,[],9223372036854775807,1,1,276,275,0,1,[]
1,Raw value forward index on start_location,trips_rawForwardIndex,0,275000,0,False,275000,57,[],9223372036854775807,1,1,276,275,0,1,[]
2,Sorted forward index with run-length encoding on start location,trips_sortedForwardIndex,0,0,0,False,275004,6,[],9223372036854775807,1,1,277,276,0,1,[]
3,Bitmap inverted index on start_location,trips_bitmapInvertedIndex_startLocation,0,0,0,False,275004,5,[],9223372036854775807,1,1,277,276,0,1,[]
4,Sorted inverted index on start_location,trips_sortedInvertedIndex_startLocation,0,0,0,False,275004,6,[],9223372036854775807,1,1,277,276,0,1,[]
5,Star Tree 1,trips_starTreeIndex,0,0,0,False,275004,5,[],9223372036854775807,1,1,277,276,0,1,[]
6,Star Tree 2,trips_starTreeIndex2,0,0,0,False,275004,5,[],9223372036854775807,1,1,277,276,0,1,[]
7,Text Index,trips_textIndex,0,0,0,False,275000,5,[],9223372036854775807,1,1,276,275,0,1,[]


Comparison of __trips_default_index (0)__ and __trips_bitmapInvertedIndex_startLocation	 (4)__:

Bitmap Inverted Index ( __trips_bitmapInvertedIndex_startLocation (5)__)
An inverted index can improve the query performance. In this case, no entries have to be scanned in filter and the query runtime is faster compared to using the dictionary encoded index (0) or a raw value forward index (1).

By using the sorted inverted index, the performance can benefit from data locality. 

In [71]:
executeSQLStatement("select AVG(payment_amount), start_location from XX GROUP BY start_location")

Top two result records of: select AVG(payment_amount), start_location from trips_default_index GROUP BY start_location


,avg(payment_amount),start_location
0,481.316816,Allmersbach im Tal
1,433.436999,Aiglsbach


Metrics of execution of: select AVG(payment_amount), start_location from XX GROUP BY start_location


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,segmentStatistics,minConsumingFreshnessTimeMs,numServersQueried,numServersResponded,numSegmentsQueried,numSegmentsProcessed,numSegmentsMatched,numConsumingSegmentsQueried,exceptions
0,Default Index (Dictionary-encoded forward index with bit compression) for each column,trips_default_index,275000,0,550000,275000,69,[],9223372036854775807,1,1,276,275,275,1,[]
1,Raw value forward index on start_location,trips_rawForwardIndex,275000,0,550000,275000,73,[],9223372036854775807,1,1,276,275,275,1,[]
2,Sorted forward index with run-length encoding on start location,trips_sortedForwardIndex,275004,0,550008,275004,57,[],9223372036854775807,1,1,277,276,276,1,[]
3,Bitmap inverted index on start_location,trips_bitmapInvertedIndex_startLocation,275004,0,550008,275004,80,[],9223372036854775807,1,1,277,276,276,1,[]
4,Sorted inverted index on start_location,trips_sortedInvertedIndex_startLocation,275004,0,550008,275004,83,[],9223372036854775807,1,1,277,276,276,1,[]
5,Star Tree 1,trips_starTreeIndex,275004,0,550008,275004,54,[],9223372036854775807,1,1,277,276,276,1,[]
6,Star Tree 2,trips_starTreeIndex2,275004,0,550008,275004,50,[],9223372036854775807,1,1,277,276,276,1,[]
7,Text Index,trips_textIndex,275000,0,550000,275000,50,[],9223372036854775807,1,1,276,275,275,1,[]


When grouping by start_location, query execution on __trips_bitmapInvertedIndex_startLocation__	 and __trips_sortedInvertedIndex_startLocation__	 can perform faster than than trips_default_index.

### Text Search Support

Support for text indexes in Pinot allows to do aribtrary search on STRING columns, for example searching for drivers whose name includes "Taylor". The metrics table only displays the table with the text index defined, as the query execution on other tables is not successful and throws exceptions instead.

In [66]:
executeSQLStatement("select * from XX WHERE TEXT_MATCH ('driver_name','Taylor') LIMIT 50000")

Top two result records of: select * from trips_textIndex WHERE TEXT_MATCH ('driver_name','Taylor') LIMIT 50000


,count,driver_name,driver_rating,end_location,end_location_state,end_zip_code,license_plate,payment_amount,payment_tip_amount,request_time_millis,rider_is_premium,rider_name,rider_rating,start_location,start_location_state,start_zip_code,trip_end_time_millis,trip_start_time_millis,trip_wait_time_millis
0,1,Ida Taylor,2,Ziltendorf Ernst-Thälmann-Siedlung,Brandenburg,15295,OZ-GB-50,473.70,28.0,1615137108610,1,Brianna Owens,1,Dreieich,Hessen,63303,1615167358586,1615138649289,1540679
1,1,Cheryl Taylor,4,Bad Berleburg Dotzlar,Nordrhein-Westfalen,57319,QG-CP-60,411.49,36.0,1586485382767,0,Charles Bryan,1,Ahlum,Sachsen-Anhalt,38489,1586503941059,1586486362396,979629


Metrics of execution of: select * from XX WHERE TEXT_MATCH ('driver_name','Taylor') LIMIT 50000


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,segmentStatistics,minConsumingFreshnessTimeMs,numServersQueried,numServersResponded,numSegmentsQueried,numSegmentsProcessed,numSegmentsMatched,numConsumingSegmentsQueried,exceptions
0,Text Index,trips_textIndex,977,0,18563,275000,507,[],9223372036854775807,1,1,276,275,261,1,[]


### Star-Tree Index
The Start-Tree index is utilizes pre-aggregation of results and is built on multiple columns. This index can improve the performance for specific queries, because the number of values to be processed is reduced by the pre-aggregation. On the one hand, a Star-Tree index has the advantage of decreased query runtime, on the other hand the required storage for the table is increased.

In [112]:
executeSQLStatement("SELECT SUM(payment_amount) FROM XX")

Top two result records of: SELECT SUM(payment_amount) FROM trips_default_index


,sum(payment_amount)
0,1.239654e+08


Metrics of execution of: SELECT SUM(payment_amount) FROM XX


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,minConsumingFreshnessTimeMs,exceptions
0,Default Index (Dictionary-encoded forward index with bit compression) for each column,trips_default_index,275000,0,275000,275000,19,9223372036854775807,[]
1,Raw value forward index on start_location,trips_rawForwardIndex,275000,0,275000,275000,17,9223372036854775807,[]
2,Sorted forward index with run-length encoding on start location,trips_sortedForwardIndex,275004,0,275004,275004,25,9223372036854775807,[]
3,Bitmap inverted index on start_location,trips_bitmapInvertedIndex_startLocation,275004,0,275004,275004,17,9223372036854775807,[]
4,Sorted inverted index on start_location,trips_sortedInvertedIndex_startLocation,275004,0,275004,275004,25,9223372036854775807,[]
5,Star Tree 1,trips_starTreeIndex,279,0,279,275004,9,1617739967634,[]
6,Star Tree 1.1,trips_starTreeIndex1,279,0,279,275004,8,1617740864126,[]
7,Star Tree 2,trips_starTreeIndex2,276,0,276,275004,15,9223372036854775807,[]
8,Text Index,trips_textIndex,275000,0,275000,275000,34,9223372036854775807,[]


When selecting the sum of payment_amount with grouping by any dimension, the start node of all dimensions will be selected.

In [113]:
executeSQLStatement("SELECT SUM(payment_amount) FROM XX WHERE rider_is_premium = 0")

Top two result records of: SELECT SUM(payment_amount) FROM trips_default_index WHERE rider_is_premium = 0


,sum(payment_amount)
0,6.213238e+07


Metrics of execution of: SELECT SUM(payment_amount) FROM XX WHERE rider_is_premium = 0


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,minConsumingFreshnessTimeMs,exceptions
0,Default Index (Dictionary-encoded forward index with bit compression) for each column,trips_default_index,137859,275000,137859,275000,33,9223372036854775807,[]
1,Raw value forward index on start_location,trips_rawForwardIndex,137859,275000,137859,275000,29,9223372036854775807,[]
2,Sorted forward index with run-length encoding on start location,trips_sortedForwardIndex,137861,275004,137861,275004,36,9223372036854775807,[]
3,Bitmap inverted index on start_location,trips_bitmapInvertedIndex_startLocation,137861,275004,137861,275004,31,9223372036854775807,[]
4,Sorted inverted index on start_location,trips_sortedInvertedIndex_startLocation,137861,275004,137861,275004,29,9223372036854775807,[]
5,Star Tree 1,trips_starTreeIndex,276,4,276,275004,22,1617739967634,[]
6,Star Tree 1.1,trips_starTreeIndex1,276,4,276,275004,16,1617740864126,[]
7,Star Tree 2,trips_starTreeIndex2,276,0,276,275004,56,9223372036854775807,[]
8,Text Index,trips_textIndex,137859,275000,137859,275000,31,9223372036854775807,[]


In [114]:
executeSQLStatement("SELECT SUM(payment_amount) FROM XX GROUP BY start_location_state")

Top two result records of: SELECT SUM(payment_amount) FROM trips_default_index GROUP BY start_location_state


,sum(payment_amount)
0,1.111922e+07
1,1.146109e+07


Metrics of execution of: SELECT SUM(payment_amount) FROM XX GROUP BY start_location_state


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,minConsumingFreshnessTimeMs,exceptions
0,Default Index (Dictionary-encoded forward index with bit compression) for each column,trips_default_index,275000,0,550000,275000,24,9223372036854775807,[]
1,Raw value forward index on start_location,trips_rawForwardIndex,275000,0,550000,275000,15,9223372036854775807,[]
2,Sorted forward index with run-length encoding on start location,trips_sortedForwardIndex,275004,0,550008,275004,19,9223372036854775807,[]
3,Bitmap inverted index on start_location,trips_bitmapInvertedIndex_startLocation,275004,0,550008,275004,13,9223372036854775807,[]
4,Sorted inverted index on start_location,trips_sortedInvertedIndex_startLocation,275004,0,550008,275004,20,9223372036854775807,[]
5,Star Tree 1,trips_starTreeIndex,4155,0,8310,275004,13,1617739967634,[]
6,Star Tree 1.1,trips_starTreeIndex1,4155,0,8310,275004,13,1617740864126,[]
7,Star Tree 2,trips_starTreeIndex2,4154,0,8308,275004,39,9223372036854775807,[]
8,Text Index,trips_textIndex,275000,0,550000,275000,32,9223372036854775807,[]


In [97]:
executeSQLStatement("select SUM(payment_amount), AVG(driver_rating), end_location from XX WHERE start_location_state = 'Hessen' and rider_is_premium=1 GROUP BY end_location")

Top two result records of: select SUM(payment_amount),end_location from trips_default_index WHERE start_location_state = 'Hessen' and rider_is_premium=1 GROUP BY end_location


,sum(payment_amount),end_location
0,487.960007,Eibelstadt
1,1307.620026,Hamburg Othmarschen


Metrics of execution of: select SUM(payment_amount),end_location from XX WHERE start_location_state = 'Hessen' and rider_is_premium=1 GROUP BY end_location


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,minConsumingFreshnessTimeMs,exceptions
0,Default Index (Dictionary-encoded forward index with bit compression) for each column,trips_default_index,4406,272026,8812,275000,16,9223372036854775807,[]
1,Raw value forward index on start_location,trips_rawForwardIndex,4406,272026,8812,275000,19,9223372036854775807,[]
2,Sorted forward index with run-length encoding on start location,trips_sortedForwardIndex,4406,271276,8812,275004,16,9223372036854775807,[]
3,Bitmap inverted index on start_location,trips_bitmapInvertedIndex_startLocation,4406,272025,8812,275004,18,9223372036854775807,[]
4,Sorted inverted index on start_location,trips_sortedInvertedIndex_startLocation,4406,271276,8812,275004,18,9223372036854775807,[]
5,Star Tree 1,trips_starTreeIndex,4401,0,8802,275004,17,9223372036854775807,[]
6,Star Tree 2,trips_starTreeIndex2,4406,272025,8812,275004,18,9223372036854775807,[]
7,Text Index,trips_textIndex,4406,272026,8812,275000,43,9223372036854775807,[]


Comparison of __trips_default_index(0)__ and __trips_starTreeIndex(5)__:


In [122]:
executeSQLStatement("select SUM(payment_amount) from XX GROUP BY start_location_state")

Top two result records of: select SUM(payment_amount) from trips_default_index GROUP BY start_location_state


,sum(payment_amount)
0,1.111922e+07
1,1.146109e+07


Metrics of execution of: select SUM(payment_amount) from XX GROUP BY start_location_state


,indextype,table,numDocsScanned,numEntriesScannedInFilter,numEntriesScannedPostFilter,totalDocs,timeUsedMs,minConsumingFreshnessTimeMs,exceptions
0,Default Index (Dictionary-encoded forward index with bit compression) for each column,trips_default_index,275000,0,550000,275000,19,9223372036854775807,[]
1,Raw value forward index on start_location,trips_rawForwardIndex,275000,0,550000,275000,19,9223372036854775807,[]
2,Sorted forward index with run-length encoding on start location,trips_sortedForwardIndex,275004,0,550008,275004,23,9223372036854775807,[]
3,Bitmap inverted index on start_location,trips_bitmapInvertedIndex_startLocation,275004,0,550008,275004,18,9223372036854775807,[]
4,Sorted inverted index on start_location,trips_sortedInvertedIndex_startLocation,275004,0,550008,275004,19,9223372036854775807,[]
5,Star Tree 1,trips_starTreeIndex,4155,0,8310,275004,19,1617739967634,[]
6,Star Tree 1.1,trips_starTreeIndex1,4155,0,8310,275004,17,1617740864126,[]
7,Star Tree 2,trips_starTreeIndex2,4154,0,8308,275004,13,9223372036854775807,[]
8,Star Tree 3,trips_starTreeIndex3,4155,0,8310,275004,92,1617741327411,[]
9,Text Index,trips_textIndex,275000,0,550000,275000,21,9223372036854775807,[]


Notizen: Preaggregation -> numDocs Scanned geringer?

For realtime tables: aggregate realtime stream as it is consumed to reduce segment sizes.
Prerequisites: all metrics should be listed in noDictionaryColumns
There should not be any multi-value dimensions
All dimension columns are treated to have a dictionary, even if they appear as noDictionary Coulmns


If the parameter numEntriesScannedPostFilter is high, this is an indicator for a low selectivity. A lot of records need to be scanned by pinot so that the query request can be answered. If numEntriesScannedPostFilter is high, using a star-tree index can help.

In [471]:
## Delete Tables
for table in table_list:
    string = "http://pinot-controller.pinot:9000/tables/" + table[0]
    response = requests.delete(string)
    print(response.json())
    table_list = []


{'status': 'Tables: [trips_default_index_REALTIME] deleted'}
{'status': 'Tables: [trips_rawForwardIndex_REALTIME] deleted'}
{'status': 'Tables: [trips_rawValueForward_index_REALTIME] deleted'}
{'status': 'Tables: [trips_bitmapInvertedIndex_REALTIME] deleted'}
{'status': 'Tables: [trips_sortedInvertedIndex_REALTIME] deleted'}
{'status': 'Tables: [trips_textIndex_REALTIME] deleted'}
